In [2]:
from pathlib import Path
import sys, os, requests, time
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
import numpy as np, pandas as pd
from kafka import KafkaConsumer, KafkaProducer
#
from data_util.stream.data_processing import processor, query_data
from data_util.stream.data_stream import kafka_streamer
%load_ext autoreload
%autoreload 2

In [3]:
class PATH_ARGS:
    path = Path('./').absolute()
    data_path = path/'data'
    log_name = 'kafka_log-movielog6_stream.csv'
    rate_log_name = log_name.split('.')[0]+'-rateonly.csv'
    #log_output_name = 'kafka_log-movielog6(Full)-cleaned.csv'
    log_fp = data_path/log_name
    
class STREAM_AGS:
    # define parameters
    topic = 'movielog6'

### Stream Data

In [19]:
print(str(PATH.log_fp))
stream = kafka_streamer(topic=topic)
stream.stream_to_csv(int(2e5), PATH.log_fp)

/home/mangyinm/m1/data/kafka_log-movielog6_stream.csv
Test message:2022-09-28T02:02:39,276853,GET /data/m/julie++julia+2009/109.mpg
done


In [ ]:
# ratings only
stream.stream_to_csv_ratings(int(1e5), PATH.data_path/PATH.rate_log_name)

#### Query User/movie

In [10]:
print(query_data(10715, category='user'))
print(query_data('next+friday+2000', category='movie'))

{'user_id': 10715, 'age': 28, 'occupation': 'executive/managerial', 'gender': 'M'}
{'id': 'next+friday+2000', 'tmdb_id': 10471, 'imdb_id': 'tt0195945', 'title': 'Next Friday', 'original_title': 'Next Friday', 'adult': 'False', 'belongs_to_collection': {'id': 43563, 'name': 'Friday Collection', 'poster_path': '/dWh1Fohgx07AEd4vRUwgIj8BYxD.jpg', 'backdrop_path': '/pXszT8ZWGj3fohU9FY0niA0IDEw.jpg'}, 'budget': '11000000', 'genres': [{'id': 35, 'name': 'Comedy'}], 'homepage': 'null', 'original_language': 'en', 'overview': 'Ice Cube returns as Craig Jones, a streetwise man from South Central Los Angeles who has a knack for getting into trouble. This time out, Craig is still trying to outsmart neighborhood bully Debo (Tommy "Tiny" Lister Jr.); after Craig gets the better of Debo in a fist fight, Debo is determined to flatten Craig in a rematch. Looking to stay out of Debo\'s way, Craig\'s dad decides that it would be a good idea for Craig to hide out with his Uncle Elroy and cousin Day-Day in

### Data Processing

In [6]:
fp = PATH_ARGS.data_path/PATH_ARGS.rate_log_name
preprocessor = processor(fp, frac=None, chunksize=5000, debug=True)
input_df = preprocessor.alt_data
print(input_df.shape)
input_df.head(2)

Size:(2825, 6), 
Columns:Index(['timestamp', 'user_id', 'request_type', 'movie_id', 'movie_time',
       'rating'],
      dtype='object')
(2825, 6)


,timestamp,user_id,request_type,movie_id,movie_time,rating
0,2022-09-28T02:13:44,107555,rate,john+carter+2012,<NA>,2
1,2022-09-28T02:13:45,312873,rate,life+is+sweet+1990,<NA>,4


In [7]:
# main pipline
input_df = preprocessor.main()
print(input_df.shape)
input_df.head(2)

Adding time info
   year  month  day  hour  weekday  user_id request_type            movie_id  \
0  2022      9   28     2        2   107555         rate    john+carter+2012   
1  2022      9   28     2        2   312873         rate  life+is+sweet+1990   

  movie_time rating  
0       <NA>      2  
1       <NA>      4  
Adding user info
   year  month  day  hour  weekday user_id request_type            movie_id  \
0  2022      9   28     2        2  107555         rate    john+carter+2012   
1  2022      9   28     2        2  312873         rate  life+is+sweet+1990   

  movie_time rating user_id age occupation gender  
0       <NA>      2  107555  27  executive      M  
1       <NA>      4  312873  27    college      F  
Adding movie info
(2825, 13)


,year,month,day,hour,weekday,user_id,request_type,movie_id,movie_time,rating,age,occupation,gender
0,2022,9,28,2,2,107555,rate,john+carter+2012,<NA>,2,27,executive,M
1,2022,9,28,2,2,312873,rate,life+is+sweet+1990,<NA>,4,27,college,F


In [ ]:
input_df.to_csv(PATH.log_fp.parent/(PATH.log_name.split('.')[0]+'-cleaned.csv'))
#

post processing

### Mongodb

In [8]:
from pyspark.sql import SparkSession
from data_util.mongo.mongo_query import mongo_ops
mongo = mongo_ops(db='movielog')

In [24]:
df = mongo.read(collection='stream-rateonly')
df = df.set_index(df.columns[1])
df = df.drop('_id', axis=1)
df.head(2)

NameError: name 'mongo' is not defined

In [30]:
#movie_df = df['movie_id'].apply(processor().expand_movie)
#movie_df = movie_df.rename({movie_df.columns[0]:'movie_id'})
movie_df = pd.concat([df['movie_id'], movie_df], axis=1)
movie_df.head(2)

,movie_id,adult,genres,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,vote_average,vote_count,message
,,,,,,,,,,,,,,
242.0,survival+quest+1988,False,"[28, 12, 53]",en,2.826282,"[3146, 8411]",[United States of America],0,96.0,[English],Released,5.8,4,NaN
8765.0,maleficent+2014,False,"[14, 12, 28, 10751, 10749]",en,19.467404,"[2, 3036, 16314]",[United States of America],758539785,97.0,[English],Released,7.0,4607,NaN


In [31]:
mongo.import_df('movie-rateonly', movie_df)

Added.


<b> Mongo Atlas

In [17]:
atlas_mongo = mongo_ops(db='movielog',host='cluster0.xuddjop.mongodb.net', 
                        username='mlproduction', password='pulp_prediction', atlas=True)

In [32]:
#atlas_mongo.import_df('stream-rateonly', df)
#atlas_mongo.import_df('user-rateonly', user_df)
atlas_mongo.import_df('movie-rateonly', movie_df)

Added.
